In [15]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd
import json
import pprint

# Machine Learning
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader, Dataset
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
import torchvision
from torchvision.transforms import v2 as TorchVisionTrns
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.ops.boxes import masks_to_boxes



# Miscellaneous
import copy
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import shutil
import time
from pathlib import Path
from skimage.io import imread

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import HTML, Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

import wandb
import utils_imri as utils
from engine import train_one_epoch, evaluate

In [16]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility (Per PyTorch Version on the same device)
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False #<! Makes things slower

In [17]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

DATA_FOLDER = os.path.join('data', 'Forehead')
TRAIN_IMAGES_FOLDER = os.path.join(DATA_FOLDER, 'train', 'images')
TRAIN_LABELS_FOLDER = os.path.join(DATA_FOLDER, 'train', 'labels')
TRAIN_MASKS_FOLDER = os.path.join(DATA_FOLDER, 'train', 'masks')
VAL_IMAGES_FOLDER = os.path.join(DATA_FOLDER, 'val', 'images')
VAL_LABELS_FOLDER = os.path.join(DATA_FOLDER, 'val', 'labels')
VAL_MASKS_FOLDER = os.path.join(DATA_FOLDER, 'val', 'masks')
TEST_IMAGES_FOLDER = os.path.join(DATA_FOLDER, 'test', 'images')
TEST_LABELS_FOLDER = os.path.join(DATA_FOLDER, 'test', 'labels')
TEST_MASKS_FOLDER = os.path.join(DATA_FOLDER, 'test', 'masks')
key_to_image_folder = {
    'train': {'images': TRAIN_IMAGES_FOLDER, 'labels': TRAIN_LABELS_FOLDER,'masks': TRAIN_MASKS_FOLDER},
    'val': {'images': VAL_IMAGES_FOLDER, 'labels': VAL_LABELS_FOLDER,'masks': VAL_MASKS_FOLDER},
    'test': {'images': TEST_IMAGES_FOLDER, 'labels': TEST_LABELS_FOLDER,'masks': TEST_MASKS_FOLDER}, 
}


T_IMG_SIZE = (480, 640, 3)

TENSOR_BOARD_BASE   = 'TB'

In [18]:
# Parameters

# Data
# numSamplesTrain = 30_000
# numSamplesVal   = 10_000
# boxFormat       = BBoxFormat.YOLO
numCls          = 2 #<! Number of classes
# maxObj          = 3

# Model
# gridSize = 5 #<! The gris is (gridSize x gridSize) 

# Training
batchSize   = 4
numWorkers  = 2 #<! Number of workers
numEpochs   = 2
λ = 20.0 #<! Localization Loss
ϵ = 0.1 #<! Label Smoothing

# Visualization
# numImg = 3


In [19]:
#auxilary functions

def only_jpg_files(files: List) -> List[str]:
    return [item for item in files if item.endswith('jpg')]

def get_bbox(filename: str, dirname: str) -> List[float]:
    json_file = os.path.join(dirname, f'{filename}.json')
    with open(json_file, 'r') as f:
        bbox = json.load(f)['bbox']
    return bbox

def get_label(filename: str, dirname: str) -> List[int]:
    json_file = os.path.join(dirname, f'{filename}.json')
    with open(json_file, 'r') as f:
        label = json.load(f)['class']
    return label

def build_dataset(batchsz,dsTrain,dsVal,dstest):

    # ADD NUM_WORKERS ETC' when moving to gpu
    train = DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchsz, collate_fn=utils.collate_fn)
    val   = DataLoader(dsVal, shuffle = False, batch_size = 2 * batchsz, collate_fn=utils.collate_fn)
    test   = DataLoader(dstest, shuffle = False, batch_size = 2 * batchsz, collate_fn=utils.collate_fn)
    
    return train, val, test

def build_network(num_classes, typ):
    if typ=='Faster RCNN':
        weights = FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
        oModel = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
        # get number of input features for the classifier
        in_features = oModel.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        oModel.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    elif typ=='Mask RCNN':
        weights = MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1
        oModel = maskrcnn_resnet50_fpn_v2(weights=weights)
        # get number of input features for the classifier
        in_features = oModel.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        oModel.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        # now get the number of input features for the mask classifier
        in_features_mask = oModel.roi_heads.mask_predictor.conv5_mask.in_channels
        hidden_layer = 256
        # and replace the mask predictor with a new one
        oModel.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer,num_classes)

    return oModel

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9,weight_decay=0.0005)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate,betas = (0.9, 0.99), weight_decay = 2e-4)
    return optimizer

In [20]:
#List of images
train_images_files = only_jpg_files(os.listdir(TRAIN_IMAGES_FOLDER))
test_images_files = only_jpg_files(os.listdir(TEST_IMAGES_FOLDER))
val_images_files = only_jpg_files(os.listdir(VAL_IMAGES_FOLDER))

## Defining the Model

using Faster R-CNN with a Resnet50 Back bone with weights from a model trained on COCO dataset

In [21]:
# Step 1: Initialize model with the best available weights
# This can be changes to different weights/Model
# suitable models can be found here https://pytorch.org/vision/stable/models.html#object-detection or
# https://pytorch.org/vision/stable/models.html#instance-segmentation
weights = FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
oModel_org = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)

In [22]:
# Model as is 

torchinfo.summary(oModel_org, (10, 3, 640, 480), col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')

Layer (type:depth-idx)                             Kernel Shape              Output Shape              Param #
FasterRCNN                                         --                        [0, 4]                    --
├─GeneralizedRCNNTransform: 1-1                    --                        [10, 3, 1088, 800]        --
├─BackboneWithFPN: 1-2                             --                        [10, 256, 17, 13]         --
│    └─IntermediateLayerGetter: 2-1                --                        [10, 2048, 34, 25]        --
│    │    └─Conv2d: 3-1                            [7, 7]                    [10, 64, 544, 400]        (9,408)
│    │    └─BatchNorm2d: 3-2                       --                        [10, 64, 544, 400]        (128)
│    │    └─ReLU: 3-3                              --                        [10, 64, 544, 400]        --
│    │    └─MaxPool2d: 3-4                         3                         [10, 64, 272, 200]        --
│    │    └─Sequential: 3-5      

In [23]:
# replace the classifier with a new one, that has
# num_classes which is user-defined
# This only changes the head

# num_classes = 2  # 1 class (person) + background
# build a network
# NNType='Faster RCNN'
NNType='Mask RCNN'
oModel=build_network(numCls,NNType)

In [24]:
# Model info of model with new head
torchinfo.summary(oModel, (10, 3, 640, 480), col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')

Layer (type:depth-idx)                             Kernel Shape              Output Shape              Param #
MaskRCNN                                           --                        [100, 4]                  --
├─GeneralizedRCNNTransform: 1-1                    --                        [10, 3, 1088, 800]        --
├─BackboneWithFPN: 1-2                             --                        [10, 256, 17, 13]         --
│    └─IntermediateLayerGetter: 2-1                --                        [10, 2048, 34, 25]        --
│    │    └─Conv2d: 3-1                            [7, 7]                    [10, 64, 544, 400]        (9,408)
│    │    └─BatchNorm2d: 3-2                       --                        [10, 64, 544, 400]        (128)
│    │    └─ReLU: 3-3                              --                        [10, 64, 544, 400]        --
│    │    └─MaxPool2d: 3-4                         3                         [10, 64, 272, 200]        --
│    │    └─Sequential: 3-5      

## Dataset and Dataloader

In [25]:
#defining dataset
class ForeheadbboxDataset_old(Dataset):

    def __init__(self, images_list, data_typ, key_folders, transform=None):
        self.images_list = images_list
        self.data_typ = data_typ
        self.transform = transform
        self.key_folders=key_folders

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):

        img=self.images_list[idx]
        file_no_ext = Path(img).stem

        #bounding box
        bbox_folder = self.key_folders[self.data_typ]['labels']
        bbox = get_bbox(file_no_ext, bbox_folder)
        bbox=torch.tensor(bbox, dtype=torch.float32)
        b=np.array([640,480,640,480])
        bbox=torch.round(bbox*b.T).to(dtype=torch.float32)

        #labels
        label=get_label(file_no_ext, bbox_folder)
        labels=torch.tensor(np.array([label]), dtype=torch.int64, device='cuda')

        #image
        image_folder = self.key_folders[self.data_typ]['images']
        full_image_filename = os.path.join(image_folder, img)
        image=torchvision.io.read_image(full_image_filename).float() / 255.0
        
        #masks
        masks_folder = self.key_folders[self.data_typ]['masks']
        full_masks_filename = os.path.join(masks_folder, img)
        mask=torchvision.io.read_image(full_masks_filename)
        
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)
        # Ensure bbox is of size (1, 4)
        if bbox.ndim == 1:
            bbox = bbox.unsqueeze(0)

        image_id = torch.tensor(np.array([idx]), dtype=torch.int64)
        area = (bbox[:, 3] - bbox[:, 1]) * (bbox[:, 2] - bbox[:,0])
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        image= torchvision.tv_tensors.Image(image)

        # Wrap sample and targets into tensors:
        target = {}
        target["boxes"]=torchvision.tv_tensors.BoundingBoxes(bbox, format="XYXY", canvas_size=TorchVisionTrns.functional.get_size(image))
        target["labels"] = labels
        # target["image_id"] = image_id
        target["image_id"] = idx
        target["area"] = area
        target["iscrowd"] = iscrowd
        target["masks"] = torchvision.tv_tensors.Mask(masks)
        if self.transform:
            image, target = self.transform(image, target)


        return image, target

In [26]:
class ForeheadbboxDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        img = read_image(img_path)
        mask = read_image(mask_path)
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set
        # of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        img = torchvision.tv_tensors.Image(img)

        target = {}
        target["boxes"] = torchvision.tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=TorchVisionTrns.functional.get_size(img))
        target["masks"] = torchvision.tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [27]:
def get_transform(train):
    transforms = []
    transforms.append(TorchVisionTrns.ToDtype(torch.float, scale=True))
    transforms.append(TorchVisionTrns.ToPureTensor())
    return TorchVisionTrns.Compose(transforms)

In [28]:
dataset = ForeheadbboxDataset('data/Forehead/train', get_transform(train=True))
dataset_test = ForeheadbboxDataset('data/Forehead/train', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])
dlTrain = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=utils.collate_fn
)

dlval = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    collate_fn=utils.collate_fn)

## Test Training


In [29]:
# Run Device
# gpu not working good on my laptop, should be changed
runDevice = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device
# runDevice = torch.device('cpu')

In [30]:
# #testing if working
# images, targets = next(iter(dlTrain))
# images = list(image.to(runDevice) for image in images)
# targets = [{k: v.to(runDevice) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
# oModel.to(runDevice)
# output = oModel(images, targets)  # Returns losses and detections
# print(output)

## Freezing all weights except the new head

In [31]:
#getting all parameters for length
params = [p for p in oModel.parameters() if p.requires_grad]
len(params)

192

In [32]:
#freezing all parameters
for param in oModel.parameters():
    param.requires_grad = False

In [33]:
#unfreezing parameters for the new head
for param in oModel.roi_heads.box_predictor.parameters():
    param.requires_grad = True
if NNType=='Mask RCNN':
    for param in oModel.roi_heads.mask_predictor.parameters():
        param.requires_grad = True

In [34]:
# parameters length
params2 = [p for p in oModel.parameters() if p.requires_grad]
len(params2)

8

## wandb

In [35]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: imrif01 (imri). Use `wandb login --relogin` to force relogin


True

In [38]:
sweep_config = {'method': 'random'}

metric = {'name': 'loss','goal': 'minimize'}
sweep_config['metric'] = metric

parameters_dict = {'optimizer': {'values': ['adam', 'sgd']}}
sweep_config['parameters'] = parameters_dict

parameters_dict.update({'epochs': {'value': 4}})

parameters_dict.update({
    'learning_rate': {'distribution': 'uniform','min': 0.0001,'max': 0.1},# a flat distribution between 0.0001 and 0.1
    'batch_size': {'values': [16]}})# integers between 2 and 16
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [16]},
                'epochs': {'value': 4},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [39]:
sweep_id = wandb.sweep(sweep_config, project="pytorch project 2")


Create sweep with ID: uufgku03
Sweep URL: https://wandb.ai/imri/pytorch%20project%202/sweeps/uufgku03


In [40]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        dataset = ForeheadbboxDataset('data/Forehead/train', get_transform(train=True))
        dataset_test = ForeheadbboxDataset('data/Forehead/train', get_transform(train=False))

        # split the dataset in train and test set
        indices = torch.randperm(len(dataset)).tolist()
        dataset = torch.utils.data.Subset(dataset, indices[:-50])
        dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])
        dlTrain = torch.utils.data.DataLoader(
        dataset,
        batch_size=2,
        shuffle=True,
        collate_fn=utils.collate_fn)


        dlVal = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=1,
        shuffle=False,
        collate_fn=utils.collate_fn)
        oModel=build_network(numCls,NNType)

        #freezing layers
        for param in oModel.roi_heads.box_predictor.parameters():
            param.requires_grad = True
        if NNType=='Mask RCNN':
            for param in oModel.roi_heads.mask_predictor.parameters():
                param.requires_grad = True
        oModel.to(runDevice)
        optimizer = build_optimizer(oModel, config.optimizer, config.learning_rate)

        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
        for epoch in range(config.epochs):
            # train for one epoch, printing every 10 iterations
            logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
            # update the learning rate
            lr_scheduler.step()
            # evaluate on the test dataset
            logger_eval=evaluate(oModel, dlVal, device=runDevice)#need to add meters from logger_eval to wandb logger, didn't get this far on CPU
            wandb.log({"loss": logger.meters['loss'].avg, "loss_classifier":logger.meters['loss_classifier'].avg,
                       "loss_box_reg":logger.meters['loss_box_reg'].avg, "loss_objectness":logger.meters['loss_objectness'].avg,
                       "loss_rpn_box_reg":logger.meters['loss_rpn_box_reg'].avg,"epoch": epoch})

In [41]:
wandb.agent(sweep_id, train, count=3)

wandb: Agent Starting Run: orr5wzc8 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 4
wandb: 	learning_rate: 0.04583988271857309
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Traceback (most recent call last):
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torchvision/models/de

Run orr5wzc8 errored:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return f

Traceback (most recent call last):
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torchvision/models/de

Run 2pzupuvk errored:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return f

Traceback (most recent call last):
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torchvision/models/de

Run pv51s4b2 errored:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "/tmp/ipykernel_230326/2519608288.py", line 41, in train
    logger=train_one_epoch(oModel, optimizer, dlTrain, runDevice, epoch, print_freq=10)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/ForeheadDetection/engine.py", line 31, in train_one_epoch
    loss_dict = model(images, targets)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/second_forehead_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return f

## Training

In [ ]:
# # # training block, nothing is truely defined here with actual thought,everything is copied from tutorial,
# # # from here wanDB and maybe gridsearch/optuna should be defined and hyper parameters searched, 
# # # maybe different optimizer and learning rate schedular, all weight are unfrozen

# # # move model to the right device
# oModel.to(runDevice)

# # # construct an optimizer
# params = [p for p in oModel.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(
#     params,
#     lr=0.005,
#     momentum=0.9,
#     weight_decay=0.0005
# )

# # and a learning rate scheduler
# lr_scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=3,
#     gamma=0.1
# )

# # # let's train it just for 2 epochs
# num_epochs = 1

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     a=train_one_epoch(oModel, optimizer, dlTrain_tst, runDevice, epoch, print_freq=10)
#     torch.save(oModel, 'BestModel.pt')
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     b=evaluate(oModel, dlVal, device=runDevice)

# print("That's it!")

lr: 0.000062  loss: 3.9202 (3.9202)  loss_classifier: 0.5476 (0.5476)  loss_box_reg: 0.0015 (0.0015)  loss_mask: 1.9977 (1.9977)  loss_objectness: 0.5192 (0.5192)  loss_rpn_box_reg: 0.8543 (0.8543)
Epoch: [0]  [ 0/88]  eta: 0:20:17  lr: 0.000062  loss: 3.9202 (3.9202)  loss_classifier: 0.5476 (0.5476)  loss_box_reg: 0.0015 (0.0015)  loss_mask: 1.9977 (1.9977)  loss_objectness: 0.5192 (0.5192)  loss_rpn_box_reg: 0.8543 (0.8543)  time: 13.8344  data: 0.2463  max mem: 2793
lr: 0.000120  loss: 3.9202 (3.9227)  loss_classifier: 0.5475 (0.5475)  loss_box_reg: 0.0015 (0.0015)  loss_mask: 1.9507 (1.9742)  loss_objectness: 0.5192 (0.5275)  loss_rpn_box_reg: 0.8543 (0.8720)
lr: 0.000177  loss: 3.9202 (3.8976)  loss_classifier: 0.5475 (0.5472)  loss_box_reg: 0.0015 (0.0015)  loss_mask: 1.9507 (1.9260)  loss_objectness: 0.5192 (0.5008)  loss_rpn_box_reg: 0.8898 (0.9221)
lr: 0.000235  loss: 3.9202 (3.9422)  loss_classifier: 0.5465 (0.5450)  loss_box_reg: 0.0014 (0.0014)  loss_mask: 1.8688 (1.9117) 

TypeError: must be real number, not NoneType

In [ ]:

# oModel.eval()
# oModel.to(runDevice)
# a=evaluate(oModel, data_loader_test, device=runDevice)